In [1]:
## 11.심층신경망 훈련 ##
import tensorflow as tf
import numpy as np


## GPU 메모리 관리 ##
#1번 방법
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #메모리 수요에 따라 할당

'''주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. 

Tensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'''

'주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. \n\nTensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'

In [31]:
tf.reset_default_graph()

n_inputs = 28 * 28 #mnist속성크기
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X") #입력층
Y = tf.placeholder(tf.int32, shape=(None), name="Y") #출력층

![초기화함수](img/초기화함수들.jpg)

In [22]:
## 11.1 세이비어 초기화와 He초기화 ##

with tf.name_scope("dnn1"):
    he_init = tf.variance_scaling_initializer()
    #tf.dense함수는 기본적으로 균등분포로 세이비어 초기화 사용, he초기화 방식으로 지정하기
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs", activation=None)
    
#He 초기화는 세이비어 초기화에서처럼 팬인과 팬아웃의 평균대신 팬인만 고려(variance_scaling_initializer의 기본값).
#매개변수로 mod="FAN_AVG"를 지정하여 바꿀수 있음

<h3>11.3.2 ELU (Exponential Linear Unit)</h3>
<p>ELU(Exponential Linear Unit)은 2105년 B.Xu et al.의 논문에서 제안된 활성화 함수이다.</p>

![nn](img/math.svg)

![elu](img/elu.png)

Elu 특
<ul>
    <li>z&lt;0 일때 음수값이 들어오므로 활성화 함수의 평균출력이 0에 더 가까워짐. 이는 앞서 이야기한 그래디언트 소실문제를 해결해줌.
    하이퍼파라메터 α는 z가 큰 음수값일 때 ELU가 수렴할 값을 정의함. 보통 1로 설정하지만 변경가능</li>
    <li>z&lt;0 이어도 그래디언트가 0이 아니므로 죽은뉴런(0만 출력하는 뉴런)을 만들지 않음</li>
    <li>α=1일 때 이 함수는 z=0에서 급격히 변동하지 않고 z=0을 포함해 모든 구간에서 매끄러워 경사하강법의 속도를 높임</li>
</ul>

In [29]:
## 11.1.2 수렴하지 않는 활성화 함수 ##
#elu함수 사용하기#

with tf.name_scope("dnn2"):
    he_init = tf.variance_scaling_initializer()
    #tf.dense함수는 기본적으로 균등분포로 세이비어 초기화 사용, he초기화 방식으로 지정하기
    #활성화 함수를 elu로 설정
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs", activation=None)

In [32]:
## leakyReLU 사용하기 ##
def leaky_relu(z, name=None): #leakyReLU는 텐서플로우에서 제공하지 않음, 직접만들어야 함
    return tf.maximum(0.01 * z, z, name=name)

with tf.name_scope("dnn3"):
    he_init = tf.variance_scaling_initializer()
    #tf.dense함수는 기본적으로 균등분포로 세이비어 초기화 사용, he초기화 방식으로 지정하기
    #활성화 함수를 laekyReLU로 설정
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=leaky_relu, kernel_initializer=he_init)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=leaky_relu, kernel_initializer=he_init)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs", activation=None)

## 1. 배치 정규화(BN, Batch Normalization)

[05-1. 심층 신경망 학습](http://excelsior-cjh.tistory.com/177)에서는 활성화 함수로는 ReLU를 사용하고 He 초기화를 통해 학습 초기 단계에서의 그래디언트 소실/폭주 문제를 줄일 수 있었지만, 이러한 문제가 학습하는 동안에 또 다시 발생할 가능성이 있다.

2015년 Sergety Ioffe와 Christian Szegedy는 ['Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift'](https://arxiv.org/pdf/1502.03167v3.pdf)라는 논문에서 **배치 정규화**(BN, Batch Normalization)를 제안했다.  배치 정규화는 각 층의 활성화 함수의 출력값 분포가 골고루 분포되도록 '강제'하는 방법으로, 각 층에서의 활성화 함수 출력값이 정규분포(normal distribution)를 이루도록 하는 방법이다. 

즉, 학습하는 동안 이전 레이어에서의 가중치 매개변수가 변함에 따라 활성화 함수 출력값의 분포가 변화하는 **내부 공변량 변화(Internal Covariate Shift) 문제를 줄이는 방법**이 바로 배치 정규화 기법이다.

배치 정규화는 아래의 그림과 같이 미니배치(mini-batch)의 데이터에서 각 feature(특성)별 평균($\mu$, mean)과 분산($\sigma^{2}$, variance)을 구한 뒤 정규화(normalize) 해준다.

![](./images/bn.png)

일반적으로 배치 정규화는 아래의 그림과 같이 Fully Connected(FC)나 Convolutional layer 바로 다음, 활성화 함수를 통과하기 전에 배치 정규화(BN)레이어를 삽입하여 사용한다.

![](./images/bn02.png)



배치 정규화는 미니배치(mini-batch)를 단위로 데이터의 분포가 평균($\mu$, mean)이 0, 분산($\sigma^{2}$, variance)이 1이 되도록 정규화(normalization)한다.  수식은 다음과 같다.

- **Input** : 미니배치 $B = \{ x_1, x_2, \dots, x_m \}$ 개의 입력 데이터, 학습 될 파라미터인 $\gamma, \beta$
- **Output** : $\{ y_i = \text{BN}_{\gamma, \beta}(x_i) \}$

$$
\begin{eqnarray*} \mu _{ B } & \leftarrow  & \frac { 1 }{ m } \sum _{ i=1 }^{ m }{ x_{ i } } & \text{// mini-batch mean}  \\ \sigma _{ B }^{ 2 } & \leftarrow  & \frac { 1 }{ m } \sum _{ i=1 }^{ m }{ \left( x_{ i }-\mu _{ B } \right) ^{ 2 } } & \text{// mini-batch variance}  \\ \hat { x } _{ i } & \leftarrow  & \frac { x_{ i }-\mu _{ B } }{ \sqrt { \sigma _{ B }^{ 2 }+\varepsilon }} & \text{// normalize} \\ y_{i} & \leftarrow & \gamma \hat{x}_{i} + \beta \equiv \text{BN}_{\gamma, \beta}(x_i) & \text{// scale and shift} \end{eqnarray*}
$$

- $\mu_B$ : 미니배치 $B$에 대한 평균
- $\sigma_B$ : 미니배치 $B$에 대한 표준편차
- $m$ : 미니배치 데이터 개수
- $\hat{x}_i$ : 평균이 0, 분산이 1로 정규화된 입력 데이터
- $\gamma$ : 정규화된 데이터에 대한 스케일(scale) 조정 파라미터
- $\beta$ : 정규화된 데이터에 대한 이동(shift) 조정 파라미터
- $\varepsilon$ : 분모가 0이 되는 것을 막기 위한 작은 숫자 ($10^{-5}$)
- $y_i$ : $\text{BN}$ 연산의 출력 결과 

### 1.1 Scale($\gamma$)과 Shift($\beta$)를 해주는 이유

위의 식에서 입력 데이터($x_i$)에 대해 정규화(normalization, $\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_{B}^{2}+\varepsilon}}$)를 하게 되면, $\hat{x}_i$의 값이 대부분 0에 가까운 값이 될 것이다. 만약, 이러한 정규화된 입력 데이터 $\hat{x}_{i}$가 시그모이드(sigmoid) 활성화 함수의 입력값으로 들어가게 되면, 비선형(nonlinearity) 함수인 sigmoid가 선형(linearity)구간에 빠지게 된다(sigmoid 함수는 0 부근에서 선형성을 띤다). 이러한 문제를 해결하기 위해서 아래의 식과 같이 정규화된 입력 데이터 $x_i$에 scaling과 shifting해주는 $\gamma$와 $\beta$를 적용 해준다.

$$
y_i = \gamma \hat{x}_i + \beta
$$


 $\gamma$와 $\beta$를 다르게 표현하면 $\hat{x}_i$에 대한 가중치($\gamma$) 와 편향(bias, $\beta$)라고 볼 수 있으며,  $\gamma$와 $\beta$는 초기값으로 $\gamma=1, \beta=0$으로 시작해(즉, 초기에는 입력값 그대로 학습), 학습 과정에서 역전파(backprop)에 의해 적합한 값으로 조정된다.

### 1.2 테스트(추론) 단계에서의 BN

테스트 단계(추론 단계처럼 데이터가 하나씩 주입된다고 가정)나 추론 단계에서는 평균($\mu_B$)과 표준편차($\sigma_{B}$)를 계산할 미니배치가 없기 때문에 전체 Training Set의 평균과 표준편차를 사용한다. 하지만, 엄청나게 많은 전체 Training set에 대한 평균과 표준편차를 계산하기에는 무리기 때문에, 아래의 식과 같이 각 $n$개의 미니배치에 대한 평균과 표준편차를 이용해 전체 Training Set의 평균과 표준편차를 대신한다.
$$
\begin{align*}
\hat{\mu} &= \frac{1}{n} \sum_{i=1}^{n}{\mu_{B}^{(i)}} \\ \hat{\sigma} &= \frac{1}{n} \sum_{i=1}^{n}{\sigma_{B}^{(i)}}
\end{align*}
$$
위와 같은 방법 대신, 모델 학습 단계에서 **지수 감소**(exponential decay) 이동 평균법(moving average)을 사용하여 평균과 표준편차를 계산할 수 있다. 

$$
\begin{align*}
\hat{\mu} & \leftarrow \alpha \hat{\mu} + \left(1 - \alpha \right)\mu_{B}^{(i)} & \text{// moving mean}\\ \hat{\sigma} & \leftarrow \alpha \hat{\sigma} + \left( 1 - \alpha \right) \sigma_{B}^{(i)} & \text{// moving stddev}
\end{align*}
$$

위의 식에서 $\alpha$는 모멘텀(momentum)값으로 일반적으로 1에 가까운 0.9, 0.99, 0.999로 설정한다.  이러한 moving mean과 moving stddev는 학습 단계에서 매 미니배치마다 업데이트 해준다.

### 1.3 BN의 장점

Batch Normalization(BN)은 논문에서 실험했던 모든 DNN의 성능이 크게 향상 시켰다. BN은 다음과 같은 장점들이 있다. 

- tanh나 sigmoid 같은 활성화 함수에 대해 그래디언트 소실(vanishing gradient)문제가 감소한다.
- 가중치 초기화에 덜 민감하다. 가중치 초기값에 크게 의존하지 않기 때문에 [05-1. 심층 신경망 학습](http://excelsior-cjh.tistory.com/177)에서 알아본 가중치 초기화 기법에 대해 크게 신경 쓰지 않아도 된다.
- 학습률(learning rate)를 크게 잡아도 gradient descent가 잘 수렴한다.
- 오버피팅을 억제한다. BN이 마치 Regularization 역할을 하기 때문에 드롭아웃(Dropout)과 같은 규제기법에 대한 필요성이 감소한다.  하지만, BN로 인한 규제는 효과가 크지 않기 때문에 드롭아웃을 함께 사용하는 것이 좋다.

### 1.4 텐서플로에서 Batch Normalization 구현하기

텐서플로(TensorFlow)에서는 [`tf.nn.batch_normalization()`](https://www.tensorflow.org/api_docs/python/tf/nn/batch_normalization)과 [`tf.layers.batch_normalization()`](https://www.tensorflow.org/api_docs/python/tf/layers/batch_normalization)을 통해 두 개의 Batch Normalization을 제공한다. 하지만, `tf.nn.batch_normalization`은 평균과 표준편차(학습할 때는 미니배치, 테스트할 때는 전체 데이터셋)를 직접 계산한 뒤 인자로 전달해줘야 하며, scaling과 shifting을 위한 $\gamma, \beta$를 만들어 줘야한다. 반면, `tf.layers.batch_normalization`은 이러한 작업들을 모두 처리해 주기 때문에 이것을 사용하는 것이 좋다. 

이제, `tf.layers.batch_normalization`을 이용하여 MNIST 데이터셋을 분류하는 DNN을 구현해 보도록 하자. DNN의 구성은 아래의 그림과 같이 784개의 입력층(input) → 300개 노드의 은닉층(hidden1) → 100개 노드의 은닉층(hidden2) → 10개의 출력층(output)으로 구성되어 있다.   

![](./images/bn03.png)


![nn](img/bn03.png)

In [86]:
## 11.1.3 배치정규화 ##

# MNIST Load
(train_x, train_y), (test_x, test_y) = tf.keras.datasets.mnist.load_data()

# Train & TestSet reshape
train_x = train_x.astype(np.float32).reshape(-1, 28*28) / 255.
train_y = train_y.astype(np.int32)
test_x = test_x.astype(np.float32).reshape(-1, 28*28) / 255.
test_y = test_y.astype(np.int32)

#Shuffle
indices = np.random.permutation(len(train_x))
train_x = train_x[indices]
train_y = train_y[indices]
indices = np.random.permutation(len(test_x))
test_x = test_x[indices]
test_y = test_y[indices]

# Split Validation set from Train set
valid_x, train_x = train_x[:5000], train_x[5000:]
valid_y, train_y = train_y[:5000], train_y[5000:]

In [87]:
def shuffle_batch(inputs, labels, batch_size):
    rnd_idx = np.random.permutation(len(inputs))
    n_batches = len(inputs) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_x, batch_y = inputs[batch_idx], labels[batch_idx]
        yield batch_x, batch_y

In [61]:
from functools import partial

tf.reset_default_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10
batch_norm_momentum = 0.9

# input layer
inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
# output layer
labels = tf.placeholder(tf.int32, shape=[None], name='labels')
# BN에 사용하기 위한 학습 유무
training = tf.placeholder_with_default(False, shape=[], name="training")

with tf.name_scope('dnn'):
    #partial()함수를 사용해서 중복되는 부분제거
    #함수를 감싸는 래퍼함수를 생성하여 매개변수의 기본값을 지정할수 있도록 함
    batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training, 
            momentum=batch_norm_momentum) #입력데이터 배치정규화

    # 1st - hidden
    hidden1 = tf.layers.dense(inputs, n_hidden1, name="hidden1")
    bn1 = batch_norm_layer(hidden1) #입력데이터 배치정규화
    # activation function
    bn1_act = tf.nn.elu(bn1)

    # 2nd - hidden
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
    bn2 = batch_norm_layer(hidden2) #입력데이터 배치정규화
    bn2_act = tf.nn.elu(bn2)

    # outputs
    logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
    logits = batch_norm_layer(logits_before_bn)
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')


learning_rate = 0.01
n_epochs = 10
batch_size = 50

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS) #이동평균을 갱신하기 위한 연산을 가져옴

#공식문서에 있는 방법, 이동평균 갱신연산을 학습연산(train op)의 종속성에 추가
with tf.control_dependencies(update_ops):
    #train_op는 update_ops의 연산이 끝난 후에 연산되게 됨
    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.minimize(xentropy)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [60]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x, 
                                          labels:batch_y, 
                                          training: True})
            
        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:03d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))
        
#정규화하는 연산때문에 속도가 느림

epoch: 000, valid. Acc: 0.9490
epoch: 001, valid. Acc: 0.9678
epoch: 002, valid. Acc: 0.9716
epoch: 003, valid. Acc: 0.9712
epoch: 004, valid. Acc: 0.9694
epoch: 005, valid. Acc: 0.9710
epoch: 006, valid. Acc: 0.9748
epoch: 007, valid. Acc: 0.9760
epoch: 008, valid. Acc: 0.9770
epoch: 009, valid. Acc: 0.9754


![nn](img/clipping.png)

In [66]:
## 11.1.4 그래디언트 클리핑 ##

tf.reset_default_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# input layer
inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name="X")
# output layer
labels = tf.placeholder(tf.int32, shape=[None], name='labels')

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(inputs, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name='hidden2')
    logits = tf.layers.dense(hidden2, n_outputs, name='logits')
    
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')

learning_rate = 0.01
threshold = 1.0
n_epochs = 10
batch_size = 50
g
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #train_op = optimizer.minimize(xentropy) #그래디언트의 계산과 적용을 동시에 수행, 나눠야할 필요 있음
    grad_and_vars = optimizer.compute_gradients(loss) # 그래디언트 계산
    # 그래디언트 클리핑, 그래디언트가 -1.0 ~ 1.0 사이가 되도록 함, 임계값은 변경할 수 있음
    clipped_grads = [(tf.clip_by_value(grad, -threshold, threshold), var)
                     for grad, var in grad_and_vars] #그래디언트가 일정 임계값을 넘어서지 못하게 잘라냄
    train_op = optimizer.apply_gradients(clipped_grads) # 클리핑 된 그래디언트 적용
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    

In [67]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x, 
                                          labels:batch_y})
            
        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:03d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

epoch: 000, valid. Acc: 0.9000
epoch: 001, valid. Acc: 0.9178
epoch: 002, valid. Acc: 0.9278
epoch: 003, valid. Acc: 0.9346
epoch: 004, valid. Acc: 0.9394
epoch: 005, valid. Acc: 0.9444
epoch: 006, valid. Acc: 0.9472
epoch: 007, valid. Acc: 0.9508
epoch: 008, valid. Acc: 0.9524
epoch: 009, valid. Acc: 0.9558


![reuse](img/reuse.png)

In [4]:
## 11.2.1 텐서플로 모델 재사용하기 ##

#재사용될 모델
tf.reset_default_graph()

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name="inputs")
labels = tf.placeholder(tf.int32, shape=None, name="labels")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(inputs, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, activation=None, name="logits")
    
with tf.name_scope("loss"):
    cross_entropy = tf.reduce_mean( #로짓을 기반으로 하는 크로스 엔트로피 계산, 0~9사이의 레이블 기대
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))
    
learning_rate = 0.01
n_epochs = 10
batch_size = 50

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(cross_entropy)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, labels, 1) #예측값이 크기순으로 1번째 안에 들면 True
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
saver = tf.train.Saver() #Saver 정의

######파라메터 저장#####
train_vars = {"inputs" : inputs,
             "labels" : labels,
             "hidden1" : hidden1,
             "hidden2" : hidden2,
             "hidden3" : hidden3,
             "hidden4" : hidden4,
             "hidden5" : hidden5,
             "logits" : logits}

for key, var in train_vars.items():
    tf.add_to_collection(key, var)
########################

#그래프 내 모든 연산 출력
for op in tf.get_default_graph().get_operations():
    print(op.name)

with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs : batch_x, labels : batch_y})
        
        #validation
        accuracy_val = accuracy.eval(feed_dict={inputs : valid_x, labels : valid_y})
        print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))
    
    save_path = saver.save(sess, "data/my_model_final.ckpt") #모델을 저장하고, 저장된 경로 반환

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
inputs
labels
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/Relu
hidden2/kernel/Initializer/random_uniform/shape
hidden2/kernel/Initializer/random_uniform/min
hidden2/kernel/Initializer/random_uniform/max
hidden2/kernel/Initializer/random_uniform/RandomUniform
hidden2/kernel/Initializer/random_uniform/sub
hidden2/kernel/Initializer/random_uniform/mul
hidden2/kernel/Initializer/random_uniform
h

epoch: 00, valid. Acc: 0.8796
epoch: 01, valid. Acc: 0.9160
epoch: 02, valid. Acc: 0.9322
epoch: 03, valid. Acc: 0.9420
epoch: 04, valid. Acc: 0.9464
epoch: 05, valid. Acc: 0.9532
epoch: 06, valid. Acc: 0.9556
epoch: 07, valid. Acc: 0.9552
epoch: 08, valid. Acc: 0.9618
epoch: 09, valid. Acc: 0.9626


In [5]:
#재사용된 모델을 사용 할 모델
#4번째 레이어만 수정

tf.reset_default_graph()

n_hidden4 = 50
n_outputs = 10


#reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]") #123층만 복원, 정규식
#restore_saver = tf.train.Saver(reuse_vars) #1-3층 복원

saver = tf.train.import_meta_graph("data/my_model_final.ckpt.meta") #저장된 그래프 불러오기
inputs = tf.get_default_graph().get_collection("inputs")[0] #텐서 복원, 방법1
#inputs = tf.get_default_graph().get_tensor_by_name("inputs:0") #텐서 복원, 방법2
#labels = tf.get_default_graph().get_collection("labels")[0]
labels = tf.get_default_graph().get_tensor_by_name("labels:0")
hidden3 = tf.get_default_graph().get_collection("hidden3")[0]
#hidden3 = tf.get_default_graph().get_tensor_by_name("hidden3:0")


#연산 복원하기
#accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0") #이름범위 안의 변수 복원
#training_op = tf.get_default_graph().get_operation_by_name("GradientDescent")

new_hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="new_hidden4")
new_logits = tf.layers.dense(new_hidden4, n_outputs, activation=None, name="new_logits")

with tf.name_scope("new_loss"):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=new_logits))

learning_rate = 0.001
n_epochs = 10
batch_size = 50

with tf.name_scope('new_train'):
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)
    
with tf.name_scope('new_eval'):
    correct = tf.nn.in_top_k(new_logits, labels, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

new_saver = tf.train.Saver()
    
# Train the New Model
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver.restore(sess, "data/my_model_final.ckpt")
    
    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(train_op, feed_dict={inputs: batch_x, 
                                          labels: batch_y})
            
        # validation
        accuracy_val = accuracy.eval(feed_dict={inputs: valid_x, labels: valid_y})
        print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))
    
    # save the new model
    save_path = new_saver.save(sess, "data/my_new_model_final.ckpt")

INFO:tensorflow:Restoring parameters from data/my_model_final.ckpt
epoch: 00, valid. Acc: 0.9624
epoch: 01, valid. Acc: 0.9602
epoch: 02, valid. Acc: 0.9704
epoch: 03, valid. Acc: 0.9734
epoch: 04, valid. Acc: 0.9702
epoch: 05, valid. Acc: 0.9760
epoch: 06, valid. Acc: 0.9726
epoch: 07, valid. Acc: 0.9790
epoch: 08, valid. Acc: 0.9758
epoch: 09, valid. Acc: 0.9766


<h2>다른 프레임워크의 모델 재사용하기</h2>
<ol>
    <li>다른 프레임워크를 사용하여 미리 훈련시킨 모델을 읽어서 재사용할 모델 파라미터를 추출.</li>
    <li>이전과 같이 모델을 만듬</li>
    <li>초기화와 관련된 연산을 구함(변수 이름에 "/Assign"이 붙어있음), 그 다음 이 연산의 두번째 입력값의 핸들을 구함</li>
    <b style="font-size:0.8em">어떤변수 v의 할당연산의 첫번째 입력은 변수 그차제, 두번째 입력은 이 변수에 할당될 값(v/initial_value)</b>
    <li>재사용 하려는 변수에 초기값 주입</li>
</ol>

In [6]:
## 11.2.2 다른 프레임워크의 모델 재사용하기 ##

tf.reset_default_graph()

n_inputs = 2
n_hidden1 = 3
original_w = [[1., 2., 3.], [4., 5., 6.]] #다른 프레임워크로부터 가중치 읽어들임
original_b = [7., 8., 9.]                 #다른 프레임워크로부터 가중치 읽어들임

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
# [...] Build the rest of the model

# Get a handle on the assignment nodes for the hidden1 variables
graph = tf.get_default_graph()
assign_kernel = graph.get_operation_by_name("hidden1/kernel/Assign")
assign_bias = graph.get_operation_by_name("hidden1/bias/Assign")
init_kernel = assign_kernel.inputs[1]
init_bias = assign_bias.inputs[1]

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init, feed_dict={init_kernel: original_w, init_bias: original_b})
    # [...] Train the model on your new task
    print(hidden1.eval(feed_dict={X: [[10.0, 11.0]]}))  # not shown in the book

[[ 61.  83. 105.]]


<h2>동결된 층 캐싱하기</h2>
<p>동결된 층은 변하지 않기 때문에 동결된 층에서 나오는 출력을 캐싱하는 것이 가능하다. 메모리가 충분하다면 전체 훈련세트에 대해 하위층을 먼저 실행할 수 있다.</p>

In [42]:
## 11.2.3 신경망의 하위층을 학습에서 제외하기 ##

tf.reset_default_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300  # 재사용
n_hidden2 = 50  # 재사용
n_hidden3 = 50  # 재사용
n_hidden4 = 20  # 새로 추가
n_outputs = 10  # 새로 추가

inputs = tf.placeholder(tf.float32, shape=[None, n_inputs], name="inputs")
labels = tf.placeholder(tf.int32, shape=[None], name="labels")

if input("방법1 사용, 방법2 사용? (1,2 입력):") == 1:
    #방법1
    with tf.name_scope("dnn"):
        hidden1 = tf.layers.dense(inputs, n_hidden1, activation=tf.nn.relu, name="hidden1") #재사용
        hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") #재사용
        hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") #재사용
        hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") #새로 추가
        new_logits = tf.layers.dense(hidden4, n_outputs, activation=None, name="new_logits") #새로 추가
else:
    #방법2
    with tf.name_scope("dnn"):
        hidden1 = tf.layers.dense(inputs, n_hidden1, activation=tf.nn.relu, name="hidden1") #재사용
        hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") #재사용
        hidden2_stop = tf.stop_gradient(hidden2) #층 동결, 이 레이어 아래의 모든층은 동결됨
        hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu, name="hidden3") #재사용
        hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") #새로 추가
        new_logits = tf.layers.dense(hidden4, n_outputs, activation=None, name="new_logits") #새로 추가

with tf.name_scope("loss"):
    cross_entropy = tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=new_logits))
    
learning_rate = 0.01
n_epochs = 10
batch_size = 50

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    ##########하위층 동결, 상위층만 훈련##########
    train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|new_logits") #3층레이어, 4층레이어, 출력층만 훈련
    training_op = optimizer.minimize(cross_entropy, var_list=train_vars) #3층레이어, 4층레이어, 출력층만 훈련할거임
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(new_logits, labels, 1) #예측값이 크기순으로 1번째 안에 들면 True
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
###상위 층 복원###
new_saver = tf.train.Saver()
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]") #레이어 1층, 레이어 2층, 레이어 3층 복구
restore_saver = tf.train.Saver(reuse_vars)

if input("캐시 사용? (y,n 입력):") == 'n':
    with tf.Session(config=config) as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        restore_saver.restore(sess, "data/my_model_final.ckpt")

        for epoch in range(n_epochs):
            for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
                sess.run(training_op, feed_dict={inputs:batch_x, labels:batch_y})

            #검증
            accuracy_val = accuracy.eval(feed_dict={inputs:valid_x, labels:valid_y})
            print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

        save_path = saver.save(sess, "data/my_new_model_final2.ckpt")

else:
    with tf.Session(config=config) as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        restore_saver.restore(sess, "data/my_model_final.ckpt")

        # Caching
        h2_cache = sess.run(hidden2, feed_dict={inputs: train_x})
        h2_cache_valid = sess.run(hidden2, feed_dict={inputs: valid_x})
        n_batches = len(train_x) // batch_size

        for epoch in range(n_epochs):
            # mini-batch for hidden2
            shuffle_idx = np.random.permutation(len(train_x))
            hidden2_batches = np.array_split(h2_cache[shuffle_idx], n_batches)
            label_batches = np.array_split(train_y[shuffle_idx], n_batches)
            for hidden2_batch, label_batch in zip(hidden2_batches, label_batches):
                sess.run(training_op, feed_dict={hidden2: hidden2_batch, 
                                              labels: label_batch})

            accuracy_val = accuracy.eval(feed_dict={hidden2: h2_cache_valid,
                                                    labels: valid_y})
            print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

        save_path = saver.save(sess, "data/my_new_model_final2.ckpt")

방법1 사용, 방법2 사용? (1,2 입력):1
캐시 사용? (y,n 입력):y
INFO:tensorflow:Restoring parameters from data/my_model_final.ckpt
epoch: 00, valid. Acc: 0.9528
epoch: 01, valid. Acc: 0.9576
epoch: 02, valid. Acc: 0.9614
epoch: 03, valid. Acc: 0.9624
epoch: 04, valid. Acc: 0.9630
epoch: 05, valid. Acc: 0.9632
epoch: 06, valid. Acc: 0.9636
epoch: 07, valid. Acc: 0.9638
epoch: 08, valid. Acc: 0.9630
epoch: 09, valid. Acc: 0.9632


<h2>11.2.7 비지도 사전훈련</h2>
<p>레이블된 데이터가 많지않다면 비지도 사전훈련을 시도할수 있다.
레이블 안된 훈련데이터가 많다면 제한된 볼츠만 머신이나 오토인코더 같은 비지도 특성추출알고리즘을 사용해 맨 하위층부터 위로 올라가면서 차례로 한층씩 학습시킬수 있다.
각 층은 훈련된 이전 층의 출력으로 훈련된다. 훈련중인 층을 제외하고 다른 층은 모두 동결된다.</p>
<br/>
<p>훈련데이터가 많지 않다면 훈련 샘플 전체를 "좋은 샘플"로 레이블하고 좋은 샘플을 오염시켜서 "나쁜 샘플"을 만들고 "나쁜 샘플"로 레이블 하는 것이다. 그런다음 좋은 샘플과 나쁜 샘플을 분류하는 첫번째 신경망을 훈련시킨다.</p>
<br/>
<p>또 다른 방법은 첫번째 신경망이 각 훈련 샘플에 대해 점수를 출력하도록 훈련시키고 좋은 샘플과 나쁜샘플의 점수차가 일정이상이 되게 하는 것이다. 이를 <b style="font-size:1.2em">최대마진학습</b>이라고 한다</p>

<h2>11.3 고속옵티마이저</h2>

<h3>&lt;&lt;모멘텀 최적화 특&gt;&gt;</h3>
<img src="img/momentum.png" alt="momentum" style="width: 300px;"/>
    <ul>
        <li>이전 그래디언트의 값을 중요하게 생각함</li>
        <li>매 반복에서 현재 그래디언트를 (학습률 η 를 곱한 후 ) 모멘텀벡터 m (momentum vector)에 더하고 이 값을 빼는 방식으로 가중치를 갱신</li>
        <li>그래디언트를 속도가 아니라 가속도로 사용</li>
        <li>일종의 마찰 저항을 표현하고 모멘텀이 너무 커지는 것을 막기 위해 알고리즘에 모멘텀(momentum)이라는 새로운 하이퍼 파라미터 β 가 등장</li>
        <li>β  : 높은 마찰 저항시 0, 마찰 저항 없음 1, 일반적인 모멘텀 값은 0.9</li>
    </ul>
    
<hr/>

<h3>&lt;&lt;네스테로프 모멘텀 최적화 특&gt;&gt;</h3>
<img src="img/nesterov.png" alt="nesterov" style="width: 400px;"/>
<img src="img/nesterov2.png" alt="nesterov" style="width: auto;"/>
    <ul>
        <li>현재 위치가 아닌 모멘텀의 방향으로 조금 앞서서 비용 함수의 그래디언트를 계산</li>
        <li>θ가 아니라  θ-βm에서 그래디언트를 측정</li>
        <li>모멘텀 최적화보다 거의 항상 훈련 속도를 높여줌</li>
    </ul>
    
<hr/>

<h3>&lt;&lt;AdaGrad 특&gt;&gt;</h3>
<img src="img/ada.png" alt="ada" style="width: 400px;"/>
<img src="img/ada2.png" alt="ada" style="width: auto;"/>
    <ul>
        <li>단계별 설명</li>
        <ol>
            <li>그래디언트 제곱을 벡터s에 누적</li>
            <li>그래디언트 벡터를 ${\sqrt{s+\epsilon}}$으로 나누어 스케일을 조정, ${\epsilon}$은 0으로 나뉘는 것을 막는 아주 작은 값</li>
        </ol>
        <li>경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소함</li>
        <li>간단한 2차 방정식 문제에 대해서는 잘 작동하지만 신경망을 훈련시킬 때 너무 일찍 멈춰버리는 경향이 있다</li>
        <li>심층신경망에는 사용하지 말아야 한다</li>
    </ul>
    
<hr/>

<h3>&lt;&lt;RMSProp 특&gt;&gt;</h3>
<img src="img/rms.png" alt="rms" style="width: 400px;"/>
    <ul>
        <li>가장 최근에 그래디언트만 누적, AdaGrad의 너무 빨리 느려지는 문제 해결</li>
        <li>일반적으로 AdaGrad보다 성능이 훨씬 좋음, 네스테로프나 모멘텀 최적화보다 빠르게 수렴</li>
    </ul>
    
<hr/>

<h3>&lt;&lt;Adam 최적화 특&gt;&gt;</h3>
<img src="img/adam.png" alt="adam" style="width: 400px;"/>
    <ul>
        <li>T는 (1부터 시작하는) 반복횟수를 나타냄</li>
        <li>학습률은 기본적으로 η = 0.001 사용</li>
    </ul>

In [3]:
## 11.3 고속 옵티마이저 ##
learning_rate = 0.01
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) #모멘텀 죄적화
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9, use_nesterov=True) #네스테로스 최적화
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate) #AdaGrad
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate, momentum=0.9, decay=0.9, epsilon=1e-10) #RMSProp
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) #Adam 최적화

In [71]:
## 11.3.6 학습률 스케줄링 ##

#학습 스케줄 구현
tf.reset_default_graph()

with tf.name_scope("train"):
    initial_learning_rate = 0.1
    decay_steps = 10000
    decay_rate = 1/10
    global_step = tf.Variable(0, trainable=False, name="global_step") #현재 훈련 반복횟수를 저장하지만 학습되지 않는 변수
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step,
                                               decay_steps, decay_rate) #10000번에 0.1씩 줄어드는 감소학습률 정의
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
    #training_op = optimizer.minimize(loss, global_step=global_step)

In [83]:
## 11.4 과대적합을 피하기 위한 규제 방법 ##
## 11.4.2 l1과 l2규제 ##

tf.reset_default_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
Y = tf.placeholder(tf.int32, shape=(None), name="Y")

<h3>릿지회귀</h3>
<img src="img/릿지회귀.png" alt="릿지회귀" style="width: 400px;"/>
<h3>라쏘회귀</h3>
<img src="img/라쏘회귀.png" alt="라쏘회귀" style="width: 400px;"/>

In [73]:
#방법1, 직접구현
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_inputs, activation=tf.nn.relu, name="hidden1")
    logits = tf.layers.dense(hidden1, n_outputs, activation=None, name="logits")

W1 = tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0") #레이어 1의 가중치를 가져옴
W2 = tf.get_default_graph().get_tensor_by_name("logits/kernel:0") #출력층의 가중치를 가져옴

scale = 0.001 #규제파라메터

#직접 규제 구현
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy") #엔트로피의 평균을 구함, 차원을 지정해서 구할수도 있음
    reg_losses = tf.reduce_sum(tf.abs(W1)) + tf.reduce_sum(tf.abs(W2))
    loss = tf.add(base_loss, scale * reg_losses, name="loss")

In [84]:
#방법2, 텐서플로우에서 제공하는 방법
scale = 0.001 #규제파라메터
my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu,
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))
    #tf.contrib.layers.l1_regularizer : l1규제
    #tf.contrib.layers.l2_regularizer : l2규제
    #tf.contrib.layers.l1_l2_regularizer : l1 l2 규제
    #변수를 생성하는 함수 (tf.get_variable,ttf.layers.dense)는 각 변수에 대한 ?_regularizer 매개변수 제공
    
with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1,name="hidden1")
    logits = my_dense_layer(hidden1, n_outputs, activation=None, name="logits")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy") #엔트로피의 평균을 구함, 차원을 지정해서 구할수도 있음
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) #손실 규제
    loss = tf.add_n([base_loss] + reg_loss, name="loss")

In [ ]:
learning_rate = 0.01
n_epochs = 30
batch_size = 50

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, Y, 1) #예측값이 크기순으로 1번째 안에 들면 True
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.Session(config=config) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_x, batch_y in shuffle_batch(train_x, train_y, batch_size):
            sess.run(training_op, feed_dict={X:batch_x, Y:batch_y})

        #검증
        if epoch % 3 == 0:
            accuracy_val = accuracy.eval(feed_dict={X:valid_x, Y:valid_y})
            print('epoch: {:02d}, valid. Acc: {:.4f}'.format(epoch, accuracy_val))

epoch: 00, valid. Acc: 0.8754
epoch: 03, valid. Acc: 0.8948
epoch: 06, valid. Acc: 0.8914
epoch: 09, valid. Acc: 0.8932
epoch: 12, valid. Acc: 0.8976
epoch: 15, valid. Acc: 0.9034
epoch: 18, valid. Acc: 0.9054
